### **Домашнее задание к лекции "Основы веб-скрапинга"**

**Обязательная часть**

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

**Дополнительная часть (необязательная)**

Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

Необходимые импорты

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import re

pd.options.display.max_colwidth = None

Список ключевых слов для запросов

In [ ]:
keyword_list = ['python', 'анализ данных', 'deepseek', 'пенза'] 

In [ ]:
def get_queries(query_list: list):

    """Функция для формирования датафрейма(ов) на основании переданного
    списка ключевых слов. 

    Returns:
        list: список датафреймов, сформированных на основании элементов переданного
            в качестве параметра списка.
    """

    url = 'https://habr.com/ru/search/'
    query_result_list = []

    # пройдемся по переданному списку ключевых слов
    for query in query_list:
        rows = []
        # передадим в параметры запроса ключевое слово
        params = {
            'q': query,
            'target_types': 'posts',
            'order': 'date'
        }

        # выполним запрос и создадим объект BeautifulSoup на основании полученного
        # ответа для дальнейшей обработки
        res = requests.get(url, params)
        time.sleep(0.2)
        soup = BeautifulSoup(res.text)

        # заберем все ссылки на первой странице выдачи в список и сделаем их 
        # полными
        refs_list = soup.find_all('a', class_='tm-title__link')

        links_list = []
        all_links = []

        for ref in refs_list:
            links_list.append(ref.get('href'))

        full_link = list(map(lambda x: 'https://habr.com' + x, links_list))

        all_links += full_link

        # используя полученный список ссылок, сделаем по ним запросы, чтобы
        # извлечь требуемые данные - дату, заголовок и саму ссылку
        for link in all_links:
            res = requests.get(link)
            time.sleep(0.2)
            soup = BeautifulSoup(res.text)
            res_datetime = soup.find(
                'span', class_='tm-article-datetime-published').find('time')['datetime']
            
            res_datetime = datetime.fromisoformat(res_datetime.replace('Z', '+00:00'))
            res_datetime = res_datetime.strftime("%d.%m.%Y")

            title = soup.find('h1', class_='tm-title tm-title_h1').find('span').text
            row = {'date':res_datetime,
                   'title': title,
                   'link': link}
            rows.append(row)

        # на основании полученного списка сформируем датафрейм и добавим его
        # в возвращаемый результат работы функции
        query_df = pd.DataFrame(rows).reset_index(drop=True)
        query_result_list.append(query_df)

    # поработаем над удалением дублей новостей, встречающихся в разных запросах

    # Колонка, по которой проверяем дубликаты
    column_to_check = 'link'

    # Собираем все значения из всех датафреймов
    all_values = pd.concat([df[column_to_check] for df in query_result_list])

    # Находим дубликаты
    duplicates = all_values[all_values.duplicated(keep=False)].unique()

    # Удаляем дубликаты из всех датафреймов, кроме первого
    for i, df in enumerate(query_result_list):
        if i > 0:  # Пропускаем первый датафрейм
            query_result_list[i] = df[~df[column_to_check].isin(duplicates)]

    return query_result_list

In [ ]:
for df in get_queries(keyword_list):
    display(df)

In [ ]:
def get_queries_advanced(query_list :list, num_pages: int = 1):
    """
    get_queries_advanced Функция для формирования датафрейма(ов) на основании переданного
    списка ключевых слов и количества нужных страниц выдачи

    Arguments:
        query_list -- список ключевых слов [str]

    Keyword Arguments:
        num_pages -- количество страниц выдачи (default: {1})

    Returns:
        список датафреймов
    """

    
    
    
    query_result_list = []

    # пройдемся по переданному списку ключевых слов
    for query in query_list:
        # передадим в параметры запроса ключевое слово
        params = {
                'q': query,
                'target_types': 'posts',
                'order': 'date'
            }
        
        rows = []

        # получим необходимое количество страниц выдачи и пройдемся по ним
        for num in range(1, num_pages+1):
            url = f'https://habr.com/ru/search/page{num}/'
                        
            # выполним запрос и создадим объект BeautifulSoup на основании полученного
            # ответа для дальнейшей обработки
            res = requests.get(url, params)
            time.sleep(0.2)
            soup = BeautifulSoup(res.text)

            # заберем все ссылки на странице выдачи в список и сделаем их 
            # полными
            refs_list = soup.find_all('a', class_='tm-title__link')

            links_list = []
            all_links = []

            for ref in refs_list:
                links_list.append(ref.get('href'))

            full_link = list(map(lambda x: 'https://habr.com' + x, links_list))

            all_links += full_link

            # используя полученный список ссылок, сделаем по ним запросы, чтобы
            # извлечь требуемые данные
            for link in all_links:
                res = requests.get(link)
                if res.status_code == 200:
                    time.sleep(0.2)
                    soup = BeautifulSoup(res.text)
                    res_datetime = soup.find('span', class_='tm-article-datetime-published').find('time')['datetime']
                    
                    res_datetime = datetime.fromisoformat(res_datetime.replace('Z', '+00:00'))
                    res_datetime = res_datetime.strftime("%d.%m.%Y")

                    title = soup.find('h1', class_='tm-title tm-title_h1').find('span').text

                    article_text = soup.find('div', class_='tm-article-body').get_text(separator='\n')

                    search_options = [
                        {'tag': 'div', 'class_': 'tm-votes-lever tm-votes-lever tm-votes-lever_appearance-article tm-article-rating__votes-switcher'},
                        {'tag': 'div', 'class_': 'tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating tm-votes-meter__value'},
                        {'tag': 'span', 'class_': 'tm-votes-meter__value'},
                    ]

                    for option in search_options:
                        reactions = soup.find(option['tag'], class_=option['class_'])  
                        if reactions is not None:  
                            reactions = soup.find(option['tag'], class_=option['class_']).get('title')
                            break
                    if reactions:    
                        pattern = r"\u2191(\d+)"  # Ищем число после стрелки вверх
                        positive_reactions = re.search(pattern, reactions).group(1)
                    else:
                        positive_reactions = 0

                    row = {'date':res_datetime,
                        'title': title,
                        'link': link,
                        'article_text': article_text,
                        'likes': positive_reactions}
                    rows.append(row)
            
        # на основании полученного списка сформируем датафрейм и добавим его
        # в возвращаемый результат работы функции
        query_df = pd.DataFrame(rows).reset_index(drop=True)
        query_result_list.append(query_df)

    # поработаем над удалением дублей новостей, встречающихся в разных запросах

    # Колонка, по которой проверяем дубликаты
    column_to_check = 'link'

    # Собираем все значения из всех датафреймов
    all_values = pd.concat([df[column_to_check] for df in query_result_list])

    # Находим дубликаты
    duplicates = all_values[all_values.duplicated(keep=False)].unique()

    # Удаляем дубликаты из всех датафреймов, кроме первого
    for i, df in enumerate(query_result_list):
        if i > 0:  # Пропускаем первый датафрейм
            query_result_list[i] = df[~df[column_to_check].isin(duplicates)]

    return query_result_list

In [ ]:
df_list = get_queries_advanced(keyword_list, 2)

for df in df_list:
    display(df)